In [ ]:
from crimson.git_pip_beta.neon_db.tester import Base, User, Package, Category, Release
from sqlalchemy.orm import sessionmaker
from sqlalchemy import (
    create_engine,
)



In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env
load_dotenv()

# Fetch variables
USER = os.getenv("user")
PASSWORD = os.getenv("password")
HOST = os.getenv("host")
PORT = os.getenv("port")
DBNAME = os.getenv("dbname")

# Construct the SQLAlchemy connection string
DATABASE_URL = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}?sslmode=require"

# Create the SQLAlchemy engine
engine = create_engine(DATABASE_URL)

# Test the connection
try:
    with engine.connect() as connection:
        print("Connection successful!")
except Exception as e:
    print(f"Failed to connect: {e}")

Connection successful!


In [8]:

Session = sessionmaker(bind=engine)


# 테이블 생성
def create_tables():
    Base.metadata.create_all(engine, checkfirst=True)
    print("테이블이 성공적으로 생성되었습니다.")


# 샘플 데이터 추가
def add_sample_data():
    session = Session()

    # 사용자 추가
    user1 = User(username="testuser1", email="user1@example.com")
    user2 = User(username="testuser2", email="user2@example.com")
    session.add_all([user1, user2])
    session.commit()

    # 패키지 추가
    package1 = Package(
        name="package1",
        repository="github.com/testuser1/package1",
        description="테스트 패키지 1",
        owner_id=user1.id,
    )

    package2 = Package(
        name="package2",
        repository="github.com/testuser2/package2",
        description="테스트 패키지 2",
        owner_id=user2.id,
    )

    session.add_all([package1, package2])
    session.commit()

    # 카테고리 추가
    category1 = Category(
        name="dev",
        module_id=package1.id,
        index_url="https://example.com/dev",
        assembled=True,
    )

    category2 = Category(
        name="prod",
        module_id=package1.id,
        index_url="https://example.com/prod",
        assembled=True,
    )

    category3 = Category(
        name="test",
        module_id=package2.id,
        index_url="https://example.com/test",
        assembled=False,
    )

    session.add_all([category1, category2, category3])
    session.commit()

    # 릴리스 추가
    release1 = Release(
        version="1.0.0", commit="abcdef1234567890", category_id=category1.id
    )

    release2 = Release(
        version="1.1.0", commit="fedcba0987654321", category_id=category1.id
    )

    release3 = Release(
        version="0.5.0", commit="1a2b3c4d5e6f7890", category_id=category3.id
    )

    session.add_all([release1, release2, release3])
    session.commit()

    print("샘플 데이터가 성공적으로 추가되었습니다.")


# 데이터 조회 테스트
def query_test():
    session = Session()

    # 모든 사용자 조회
    print("\n--- 모든 사용자 ---")
    users = session.query(User).all()
    for user in users:
        print(f"사용자: {user.username}, 이메일: {user.email}")

    # 모든 패키지 조회
    print("\n--- 모든 패키지 ---")
    packages = session.query(Package).all()
    for package in packages:
        print(
            f"패키지: {package.name}, 저장소: {package.repository}, 소유자: {package.owner.username}"
        )

    # 특정 사용자의 패키지 조회
    print("\n--- testuser1의 패키지 ---")
    user1_packages = (
        session.query(Package).join(User).filter(User.username == "testuser1").all()
    )
    for package in user1_packages:
        print(f"패키지: {package.name}, 저장소: {package.repository}")

    # 카테고리 및 릴리스 조회
    print("\n--- 카테고리 및 릴리스 ---")
    categories = session.query(Category).all()
    for category in categories:
        print(
            f"카테고리: {category.name}, 모듈: {category.module.name}, Assembled: {category.assembled}"
        )

        # 각 카테고리의 릴리스 출력
        for release in category.releases:
            print(f"  릴리스: {release.version}, 커밋: {release.commit}")


if __name__ == "__main__":
    # 테이블 생성
    create_tables()

    # 샘플 데이터 추가
    add_sample_data()

    # 데이터 조회 테스트
    query_test()


테이블이 성공적으로 생성되었습니다.
샘플 데이터가 성공적으로 추가되었습니다.

--- 모든 사용자 ---
사용자: testuser1, 이메일: user1@example.com
사용자: testuser2, 이메일: user2@example.com

--- 모든 패키지 ---
패키지: package1, 저장소: github.com/testuser1/package1, 소유자: testuser1
패키지: package2, 저장소: github.com/testuser2/package2, 소유자: testuser2

--- testuser1의 패키지 ---
패키지: package1, 저장소: github.com/testuser1/package1

--- 카테고리 및 릴리스 ---
카테고리: dev, 모듈: package1, Assembled: True
  릴리스: 1.0.0, 커밋: abcdef1234567890
  릴리스: 1.1.0, 커밋: fedcba0987654321
카테고리: prod, 모듈: package1, Assembled: True
카테고리: test, 모듈: package2, Assembled: False
  릴리스: 0.5.0, 커밋: 1a2b3c4d5e6f7890
